In [ ]:
#%load_ext autotime
import time

In [1]:
import pandas as pd
df = pd.read_csv('Consumer_Complaints.csv')
# df = df.head(n=10000)

C:\Users\Tami\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


נרמול - המאגר - הוצאת הרשומות הרלוונטיות  

In [2]:

from io import StringIO

col = ['Product', 'Consumer complaint narrative']
df = df[col]
df = df[pd.notnull(df['Consumer complaint narrative'])]
df.columns = ['Product', 'Consumer complaint narrative']
df['category_id'] = df['Product'].factorize()[0]

df.head()

,Product,Consumer complaint narrative,category_id
225,Debt collection,Im in complete shock about this negative item ...,0
249,Debt collection,This so called receivable performance Fraudule...,0
287,"Credit reporting, credit repair services, or o...",Creditor Information MONTEREY COLL SVC XXXX XX...,1
331,Debt collection,Elite Financial has contacted myself and my fi...,0
534,"Credit reporting, credit repair services, or o...",I had sought to have some erroneous reporting ...,1


יצירת טבלה ומילון לנושאים + תז

In [3]:
category_id_df = df[['Product', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Product']].values)

category_id_df

,Product,category_id
225,Debt collection,0
287,"Credit reporting, credit repair services, or o...",1
875,Mortgage,2
1007,Credit card or prepaid card,3
1361,Checking or savings account,4
2441,"Payday loan, title loan, or personal loan",5
2446,"Money transfer, virtual currency, or money ser...",6
3417,Student loan,7
3724,Vehicle loan or lease,8
413876,Credit reporting,9


הצגת דיאגרמת שכיחויות

In [4]:
import pandas as pd
import numpy as np 
import nltk
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
#drop nulls, lowercase, stemming, stop_words, includes_same_labeks, 

def preprocessing(df):
    
    df = df.loc[df['Consumer complaint narrative'] > ' ' ,['Consumer complaint narrative', 'Product']]
    df['Consumer complaint narrative'] = df['Consumer complaint narrative'].map(lambda x: x.lower())
    
    product = []
    switches = {'Credit reporting' :'Credit reporting, credit repair services, or other personal consumer reports',
           'Prepaid card':'Credit card or prepaid card',
           'Credit card':'Credit card or prepaid card' ,
           'Payday loan' :'Payday loan, title loan, or personal loan' ,
           'Money transfers' :'Money transfer, virtual currency, or money service',
           'Virtual currency':'Money transfer, virtual currency, or money service' }
    for row in df['Product']:
        product.append(switches.get(row, row))
    df['Product'] = product

    ps = PorterStemmer()
    texts = []
    for row in df['Consumer complaint narrative']:
        stem_sent = []
        for word in row.split():
            stem_sent.append(ps.stem(word))
            stem_sent.append(' ')
        stem_sent =  ''.join(stem_sent)   
        texts.append(stem_sent)
    df['Consumer complaint narrative'] = texts
    
    punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%", 'xxxx', 'xx/xx/']
    stop_words = text.ENGLISH_STOP_WORDS.union(punc)
    desc = df['Consumer complaint narrative'].values
    vectorizer = TfidfVectorizer(stop_words = stop_words, max_features=10000)
    features = vectorizer.fit_transform(desc)
    print(features.shape)
    return features
        

In [ ]:
# !pip install -U imbalanced-learn

In [6]:
 from imblearn.over_sampling import RandomOverSampler

# df = RandomOverSampler(df)

In [7]:

# df = df.head(200000)
# features = preprocessing(df)


In [8]:
import pickle
# with open('features_all.pickle', 'wb') as file:
#     pickle.dump(features, file)

In [9]:
with open('features_all.pickle', 'rb') as file:
    features = pickle.load(file)

In [ ]:
labels = df.category_id
# labels =labels.head(100000)

In [11]:
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter
from sklearn.svm import LinearSVC

X_resampled, y_resampled = SMOTE().fit_resample(features, labels)
# print(sorted(Counter(y_resampled).items()))
clf_smote = LinearSVC().fit(X_resampled, y_resampled)

In [12]:
y_pred = clf_smote.predict(X_resampled)
import sklearn.metrics as matric
matric.accuracy_score (y_resampled, y_pred)

0.9156856935803118

In [ ]:
X_resampled, y_resampled = ADASYN().fit_resample(features, labels)
print(sorted(Counter(y_resampled).items()))
clf_adasyn = LinearSVC().fit(X_resampled, y_resampled)



In [ ]:
y_pred = clf_smote.predict(X_resampled)
import sklearn.metrics as matric
matric.accuracy_score (y_resampled, y_pred)


In [ ]:
# from sklearn.feature_selection import chi2
# import numpy as np
# N = 2
# for Product, category_id in sorted(category_to_id.items()):
#     features_chi2 = chi2(features, labels == category_id)
#     indices = np.argsort(features_chi2[0])
#     feature_names = np.array(tfidf.get_feature_names())[indices]
#     unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
#     bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
#     print("# '{}':".format(Product))
#     print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
#     print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
    

In [1]:

from sklearn.naive_bayes import MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
#     MLPClassifier(alpha=1),
#     RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
     LinearSVC(),
#     MultinomialNB(),
     LogisticRegression(random_state=0),
    
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [ ]:
cv_df.groupby('model_name').accuracy.mean()


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(alpha=1)
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.2, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# conf_mat = confusion_matrix(y_test, y_pred)
# fig, ax = plt.subplots(figsize=(10,10))
# sns.heatmap(conf_mat, annot=True, fmt='d', xticklabels=category_id_df.Product.values, yticklabels=category_id_df.Product.values)
# plt.ylabel('Actual')
# plt.xlabel('Predicted')
# plt.show()

In [ ]:
import sklearn.metrics as matric
matric.accuracy_score (y_test, y_pred)


In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['Consumer_complaint_narrative'], df['Product'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)